In [1]:
from elasticsearch import Elasticsearch
import pandas as pd

es = Elasticsearch("http://localhost:9200", basic_auth=("elastic", "NiTcEgNphln17-PjmS_f"))

response = es.search(
    index="app-logs",
    size=1000,
     body={
        "query": {
            "exists": {
            "field" : "geoip"
            }
        }
           },
)

data = [doc['_source'] for doc in response['hits']['hits']]
df = pd.DataFrame(data)
print(df.head(10))

                ip                @timestamp  \
0     144.8.75.189  2025-06-17T10:15:18.000Z   
1    24.85.243.248  2025-06-17T12:28:08.000Z   
2    19.56.241.190  2025-06-17T11:47:45.000Z   
3      149.82.22.8  2025-06-17T06:11:04.000Z   
4    82.161.58.192  2025-06-17T13:08:33.000Z   
5  144.164.194.204  2025-06-17T11:31:16.000Z   
6    141.22.41.210  2025-06-17T13:11:56.000Z   
7     132.7.239.51  2025-06-17T06:37:05.000Z   
8    163.64.80.149  2025-06-17T07:33:34.000Z   
9   136.85.152.234  2025-06-17T13:47:29.000Z   

                                                 log  \
0  {'file': {'path': 'E:/Downloads/application_lo...   
1  {'file': {'path': 'E:/Downloads/application_lo...   
2  {'file': {'path': 'E:/Downloads/application_lo...   
3  {'file': {'path': 'E:/Downloads/application_lo...   
4  {'file': {'path': 'E:/Downloads/application_lo...   
5  {'file': {'path': 'E:/Downloads/application_lo...   
6  {'file': {'path': 'E:/Downloads/application_lo...   
7  {'file': {'path': 'E

C:\Users\Ojas Gandheel\AppData\Local\Temp\ipykernel_11240\1930121871.py:6: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(


In [3]:
numeric_cols=df.select_dtypes(include=['int64' , 'float64']).columns
categorical_cols = df.select_dtypes(include=['object','category','bool']).columns                              
print(numeric_cols)
print(categorical_cols)

Index([], dtype='object')
Index(['ip', '@timestamp', 'log', 'geoip', '@version', 'user_id', 'host',
       'level', 'timestamp', 'event', 'service', 'message'],
      dtype='object')


In [5]:
del df['@timestamp']

KeyError: '@timestamp'

In [6]:
df['timestamp']=pd.to_datetime(df['timestamp'])

In [7]:
print(df['timestamp'])

0     2025-06-17 15:45:18
1     2025-06-17 17:58:08
2     2025-06-17 17:17:45
3     2025-06-17 11:41:04
4     2025-06-17 18:38:33
              ...        
995   2025-06-17 14:35:56
996   2025-06-17 14:46:42
997   2025-06-17 19:53:07
998   2025-06-17 17:23:31
999   2025-06-17 15:40:39
Name: timestamp, Length: 1000, dtype: datetime64[ns]


In [8]:
def sanitize_column(col, keep='all'):
    def extract(x):
        if isinstance(x, dict):
            return str(x) if keep == 'all' else str(next(iter(x.values()), ''))
        elif isinstance(x, list):
            return str(x) if keep == 'all' else str(x[0]) if x else ''
        elif pd.isna(x):
            return ''
        else:
            return str(x)
    return col.apply(extract)

In [9]:
df_clean = df.apply(sanitize_column)

In [10]:
numeric_cols=df_clean.select_dtypes(include=['int64' , 'float64']).columns
categorical_cols = df_clean.select_dtypes(include=['object','category','bool']).columns                              
print(numeric_cols)
print(categorical_cols)

Index([], dtype='object')
Index(['ip', 'log', 'geoip', '@version', 'user_id', 'host', 'level',
       'timestamp', 'event', 'service', 'message'],
      dtype='object')


In [10]:
print(df_clean.head(10))

                ip                                                log  \
0     144.8.75.189  {'file': {'path': 'E:/Downloads/application_lo...   
1    24.85.243.248  {'file': {'path': 'E:/Downloads/application_lo...   
2    19.56.241.190  {'file': {'path': 'E:/Downloads/application_lo...   
3      149.82.22.8  {'file': {'path': 'E:/Downloads/application_lo...   
4    82.161.58.192  {'file': {'path': 'E:/Downloads/application_lo...   
5  144.164.194.204  {'file': {'path': 'E:/Downloads/application_lo...   
6    141.22.41.210  {'file': {'path': 'E:/Downloads/application_lo...   
7     132.7.239.51  {'file': {'path': 'E:/Downloads/application_lo...   
8    163.64.80.149  {'file': {'path': 'E:/Downloads/application_lo...   
9   136.85.152.234  {'file': {'path': 'E:/Downloads/application_lo...   

                                               geoip @version user_id  \
0  {'timezone': 'America/Chicago', 'country_code3...        1  u73384   
1  {'timezone': 'America/Vancouver', 'region_name.

In [11]:
import time
from ipwhois import IPWhois
import pandas as pd 

def trace_ip(ip):
    try:
        obj = IPWhois(ip)
        results = obj.lookup_whois()
        return results
    except Exception as e:
        return f"Error: {str(e)}"

def display_trace_info(ipp):
    info = trace_ip(ipp)
    if isinstance(info, dict):
      return {
            "ip": ipp,
            "asn": info.get('asn' , ''),
            "asn_country": info.get('asn_country_code' , ''),
            "asn_description": info.get('asn_description' , '')
        }
    else:
            print(info)

results = []

for idx, x in enumerate(df_clean['ip']):
    print(f"Processing {idx+1}/{len(df_clean['ip'])}: {x}")
    ip_info = display_trace_info(x)
    print(ip_info)
    results.append(ip_info)
      



Processing 1/1000: 144.8.75.189
{'ip': '144.8.75.189', 'asn': 'NA', 'asn_country': 'US', 'asn_description': 'NA'}
Processing 2/1000: 24.85.243.248
{'ip': '24.85.243.248', 'asn': '6327', 'asn_country': 'CA', 'asn_description': 'SHAW, CA'}
Processing 3/1000: 19.56.241.190
{'ip': '19.56.241.190', 'asn': 'NA', 'asn_country': 'US', 'asn_description': 'NA'}
Processing 4/1000: 149.82.22.8
{'ip': '149.82.22.8', 'asn': 'NA', 'asn_country': 'US', 'asn_description': 'NA'}
Processing 5/1000: 82.161.58.192
{'ip': '82.161.58.192', 'asn': '3265', 'asn_country': 'NL', 'asn_description': 'XS4ALL-NL Amsterdam, NL'}
Processing 6/1000: 144.164.194.204
{'ip': '144.164.194.204', 'asn': '224', 'asn_country': 'NO', 'asn_description': 'UNINETT UNINETT, The Norwegian University & Research Network, NO'}
Processing 7/1000: 141.22.41.210
{'ip': '141.22.41.210', 'asn': '680', 'asn_country': 'DE', 'asn_description': 'DFN Verein zur Foerderung eines Deutschen Forschungsnetzes e.V., DE'}
Processing 8/1000: 132.7.239.5

In [13]:
print(results)

[{'ip': '144.8.75.189', 'asn': 'NA', 'asn_country': 'US', 'asn_description': 'NA'}, {'ip': '24.85.243.248', 'asn': '6327', 'asn_country': 'CA', 'asn_description': 'SHAW, CA'}, {'ip': '19.56.241.190', 'asn': 'NA', 'asn_country': 'US', 'asn_description': 'NA'}, {'ip': '149.82.22.8', 'asn': 'NA', 'asn_country': 'US', 'asn_description': 'NA'}, {'ip': '82.161.58.192', 'asn': '3265', 'asn_country': 'NL', 'asn_description': 'XS4ALL-NL Amsterdam, NL'}, {'ip': '144.164.194.204', 'asn': '224', 'asn_country': 'NO', 'asn_description': 'UNINETT UNINETT, The Norwegian University & Research Network, NO'}, {'ip': '141.22.41.210', 'asn': '680', 'asn_country': 'DE', 'asn_description': 'DFN Verein zur Foerderung eines Deutschen Forschungsnetzes e.V., DE'}, {'ip': '132.7.239.51', 'asn': '721', 'asn_country': 'US', 'asn_description': 'DNIC-ASBLK-00721-00726, US'}, {'ip': '163.64.80.149', 'asn': 'NA', 'asn_country': 'FR', 'asn_description': 'NA'}, {'ip': '136.85.152.234', 'asn': 'NA', 'asn_country': 'US', '

In [20]:
clean_results = [r for r in results if r is not None]

In [21]:
print(clean_results)

[{'ip': '144.8.75.189', 'asn': 'NA', 'asn_country': 'US', 'asn_description': 'NA'}, {'ip': '24.85.243.248', 'asn': '6327', 'asn_country': 'CA', 'asn_description': 'SHAW, CA'}, {'ip': '19.56.241.190', 'asn': 'NA', 'asn_country': 'US', 'asn_description': 'NA'}, {'ip': '149.82.22.8', 'asn': 'NA', 'asn_country': 'US', 'asn_description': 'NA'}, {'ip': '82.161.58.192', 'asn': '3265', 'asn_country': 'NL', 'asn_description': 'XS4ALL-NL Amsterdam, NL'}, {'ip': '144.164.194.204', 'asn': '224', 'asn_country': 'NO', 'asn_description': 'UNINETT UNINETT, The Norwegian University & Research Network, NO'}, {'ip': '141.22.41.210', 'asn': '680', 'asn_country': 'DE', 'asn_description': 'DFN Verein zur Foerderung eines Deutschen Forschungsnetzes e.V., DE'}, {'ip': '132.7.239.51', 'asn': '721', 'asn_country': 'US', 'asn_description': 'DNIC-ASBLK-00721-00726, US'}, {'ip': '163.64.80.149', 'asn': 'NA', 'asn_country': 'FR', 'asn_description': 'NA'}, {'ip': '136.85.152.234', 'asn': 'NA', 'asn_country': 'US', '

In [22]:
import pandas as pd
z = pd.DataFrame(clean_results)
print(z)

                  ip    asn asn_country  \
0       144.8.75.189     NA          US   
1      24.85.243.248   6327          CA   
2      19.56.241.190     NA          US   
3        149.82.22.8     NA          US   
4      82.161.58.192   3265          NL   
..               ...    ...         ...   
979    83.178.97.127   1257          SE   
980   106.89.207.248   4134          CN   
981   205.200.84.128  54937          CA   
982    47.138.171.79   5650          US   
983  114.241.111.181   4808          CN   

                                       asn_description  
0                                                   NA  
1                                             SHAW, CA  
2                                                   NA  
3                                                   NA  
4                              XS4ALL-NL Amsterdam, NL  
..                                                 ...  
979                                          TELE2, EU  
980        CHINANET-BACKBON

In [27]:
z.to_csv("ans_data.csv" , index=False)

In [29]:
import pandas as pd
df_clean = pd.concat([df_clean , z] , axis=1)

In [30]:
print(df_clean.head())

              ip                                                log  \
0   144.8.75.189  {'file': {'path': 'E:/Downloads/application_lo...   
1  24.85.243.248  {'file': {'path': 'E:/Downloads/application_lo...   
2  19.56.241.190  {'file': {'path': 'E:/Downloads/application_lo...   
3    149.82.22.8  {'file': {'path': 'E:/Downloads/application_lo...   
4  82.161.58.192  {'file': {'path': 'E:/Downloads/application_lo...   

                                               geoip @version user_id  \
0  {'timezone': 'America/Chicago', 'country_code3...        1  u73384   
1  {'timezone': 'America/Vancouver', 'region_name...        1  u53732   
2  {'timezone': 'America/Chicago', 'country_code3...        1  u83154   
3  {'timezone': 'America/Chicago', 'country_code3...        1  u48279   
4  {'timezone': 'Europe/Amsterdam', 'region_name'...        1  u95437   

                               host  level            timestamp  \
0  {'name': 'Ojas-Laptop-I56L035J'}   INFO  2025-06-17 15:45:18   
